<a href="https://colab.research.google.com/github/DarioneNazionale/KickLearning/blob/main/working_on_data/merging_storic_information.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Merging the historic stada in the dataset

### Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [4]:
historic_data_path = "./drive/MyDrive/Project/Data"
data_path = "./drive/MyDrive/Project/Data/datasets"
destination_path = "./drive/MyDrive/Project/Data/complete"

In [2]:
import os
from os.path import join
import pandas as pd
import time

### Fetching the historic data

In [ ]:
storic_df = pd.read_csv(join(historic_data_path, "complete_storic_info.csv"))

### Historic Info Function
Function to be then applied on all the dataset

In [ ]:
def get_history(entry, storic_df):
    reduced_storic_df = storic_df[(storic_df["creator_id"] == entry["creator_id"])&&(storic_df["id"] != entry["id"])&&(storic_df["time"] < entry["launched_at"])]
    reduced_storic_df = reduced_storic_df.iloc[reduced_storic_df.groupby('id')['time'].idxmax()]
    reduced_storic_df = reduced_storic_df.groupby('creator_id').apply(list)
    
    return reduced_storic_df.drop(columns=["id", "creator_id"])

## Applying on all the dataset
Merging the historic data with the actual one file by file

In [ ]:
file_list = sorted(filter(lambda file: file[:4]=="file", os.listdir("/content/drive/MyDrive/Second Semester/SL/Project/Data/datasets_bakup")), reverse=True)

start = time.perf_counter()
for i, file in enumerate(file_list):

    df = pd.read_csv(os.path.join(data_path, file), usecols=["id", "creator_id", "year"], index_col=[0])

    df_history = df.apply(lambda entry: get_history(entry, storic_df))

    df = pd.concat((df, df_history), axis=1)

    df.to_csv(join(destination_path, file), index=False)

    if i>0 and i % round(len(file_list)/5) == 0:
        print(f"Time elapsed: {time.perf_counter()-start}; remaning time: {(time.perf_counter()-start)/i*(len(file_list)-i)}")
        print(f"Done {round(i/len(file_list)*100)}% untill now, in total {i} files", end="\n\n")

print(f"Time elapsed in total: {time.perf_counter()-start}")